In [1]:
filepath=r"C:\Users\USER\OneDrive\Documents\draw football model datasets\combined_draw.csv"

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import wandb
from pathlib import Path

In [3]:
def load_and_preprocess_data(filepath):
    """Load and preprocess the football match data"""
    data = pd.read_csv(filepath)
    
    print("Dataset Info:")
    print(data.info())
    print(f"\nDuplicates found: {data.duplicated().any()}")
    print(f"Dataset shape: {data.shape}")
    
    # Create a copy for processing
    data2 = data.copy()
    
    # Feature engineering - create win/draw/loss indicators
    data2['home_wins'] = (data2['Result'] == 'HOME_TEAM').astype(int)
    data2['home_draws'] = (data2['Result'] == 'DRAW').astype(int)
    data2['home_losses'] = (data2['Result'] == 'AWAY_TEAM').astype(int)
    
    data2['away_wins'] = (data2['Result'] == 'AWAY_TEAM').astype(int)
    data2['away_draws'] = (data2['Result'] == 'DRAW').astype(int)
    data2['away_losses'] = (data2['Result'] == 'HOME_TEAM').astype(int)
    
    # Create cumulative statistics
    data2['Cumulative_Home_Wins'] = data2.groupby('home_team')['home_wins'].cumsum()
    data2['Cumulative_Home_Draws'] = data2.groupby('home_team')['home_draws'].cumsum()
    data2['Cumulative_Home_Losses'] = data2.groupby('home_team')['home_losses'].cumsum()
    
    data2['Cumulative_Away_Wins'] = data2.groupby('away_team')['away_wins'].cumsum()
    data2['Cumulative_Away_Draws'] = data2.groupby('away_team')['away_draws'].cumsum()
    data2['Cumulative_Away_Losses'] = data2.groupby('away_team')['away_losses'].cumsum()
    
    # Encode categorical variables
    data2['home_team'] = data2['home_team'].astype("category").cat.codes
    data2['away_team'] = data2['away_team'].astype("category").cat.codes
    data2["Result"] = data2["Result"].astype("category").cat.codes
    
    # Prepare features and target
    X = data2.drop(columns=["id", "matchday", "date", "home_score", "away_score", "Result", 
                           "home_points", "away_points", "home_wins", "home_draws", "home_losses",
                           "away_wins", "away_draws", "away_losses"], axis=1)
    y = data2["Result"]
    
    # Handle missing values
    X = X.fillna(0)
    
    return X, y

In [4]:
class MatchPredictionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, out_dim, dropout_prob=0.5, num_layers=2):
        super(MatchPredictionModel, self).__init__()
        
        layers = []
        current_dim = input_dim
        
        for i in range(num_layers):
            next_dim = hidden_dim // (2 ** i)
            layers.extend([
                nn.Linear(current_dim, next_dim),
                nn.BatchNorm1d(next_dim),
                nn.ReLU(),
                nn.Dropout(dropout_prob)
            ])
            current_dim = next_dim
        
        layers.append(nn.Linear(current_dim, out_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [5]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

def validate_epoch(model, test_loader, criterion, device):
    """Validate the model"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_predictions = []
    all_targets = []
    all_probs = []
    
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            probs = torch.softmax(outputs, dim=1)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(batch_y.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy, all_predictions, all_targets, all_probs

In [6]:
def compute_feature_importance(model, X_tensor, feature_names, device, num_samples=50):
    """Compute feature importance using gradient-based analysis"""
    model.eval()
    X_sample = X_tensor[:num_samples].to(device)
    X_sample.requires_grad_(True)
    
    outputs = model(X_sample)
    
    class_gradients = []
    for class_idx in range(3):
        model.zero_grad()
        class_output = outputs[:, class_idx].sum()
        class_output.backward(retain_graph=True)
        class_gradients.append(X_sample.grad.abs().mean(0).cpu().detach().numpy())
    
    feature_importance = np.mean(class_gradients, axis=0)
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importance
    }).sort_values('Importance', ascending=False)
    
    return importance_df

In [7]:

def train_model(config=None):
    """Main training function with W&B integration"""
    
    # Initialize W&B run
    with wandb.init(config=config):
        config = wandb.config
        
        # Set device
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")
        
        # Load and preprocess data
        X, y = load_and_preprocess_data(config.data_path)
        
        # Standardize features
        sc = StandardScaler()
        X_scaled = sc.fit_transform(X)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Convert to tensors
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.LongTensor(y_train.values)
        X_test_tensor = torch.FloatTensor(X_test)
        y_test_tensor = torch.LongTensor(y_test.values)
        
        # Create data loaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)
        
        # Initialize model
        model = MatchPredictionModel(
            input_dim=X_train_tensor.shape[1],
            hidden_dim=config.hidden_dim,
            out_dim=3,
            dropout_prob=config.dropout_prob,
            num_layers=config.num_layers
        ).to(device)
        
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        
        if config.optimizer == "Adam":
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, 
                                  weight_decay=config.weight_decay)
        elif config.optimizer == "SGD":
            optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, 
                                 momentum=0.9, weight_decay=config.weight_decay)
        else:
            optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate,
                                   weight_decay=config.weight_decay)
        
        # Learning rate scheduler
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5, verbose=True
        )
        
        # Watch model with W&B
        wandb.watch(model, criterion, log="all", log_freq=10)
        
        # Log dataset info
        wandb.log({
            "dataset/train_size": len(X_train),
            "dataset/test_size": len(X_test),
            "dataset/num_features": X_train.shape[1],
            "dataset/class_distribution": dict(y_train.value_counts())
        })
        
        # Training loop
        best_val_accuracy = 0
        best_model_path = "best_model.pth"
        patience_counter = 0
        
        for epoch in range(config.epochs):
            # Train
            train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
            
            # Validate
            val_loss, val_acc, val_preds, val_targets, val_probs = validate_epoch(
                model, test_loader, criterion, device
            )
            
            # Update scheduler
            scheduler.step(val_loss)
            
            # Log metrics
            metrics = {
                "epoch": epoch,
                "train/loss": train_loss,
                "train/accuracy": train_acc,
                "val/loss": val_loss,
                "val/accuracy": val_acc,
                "learning_rate": optimizer.param_groups[0]['lr']
            }
            wandb.log(metrics)
            
            # Save best model
            if val_acc > best_val_accuracy:
                best_val_accuracy = val_acc
                patience_counter = 0
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'val_accuracy': val_acc,
                    'val_loss': val_loss,
                    'config': dict(config)
                }, best_model_path)
                
                # Log model as W&B artifact
                artifact = wandb.Artifact(
                    name=f"model-{wandb.run.id}",
                    type="model",
                    description=f"Best model at epoch {epoch} with val_acc {val_acc:.2f}%"
                )
                artifact.add_file(best_model_path)
                wandb.log_artifact(artifact)
                
            else:
                patience_counter += 1
                if patience_counter >= config.patience:
                    print(f"Early stopping at epoch {epoch}")
                    break
            
            # Periodic detailed logging
            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{config.epochs}]")
                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
                print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
                print("-" * 50)
        
        # Load best model for final evaluation
        checkpoint = torch.load(best_model_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        
        # Final evaluation
        final_val_loss, final_val_acc, final_preds, final_targets, final_probs = validate_epoch(
            model, test_loader, criterion, device
        )
        
        # Classification report
        class_names = ["Away Win", "Draw", "Home Win"]
        report = classification_report(final_targets, final_preds, 
                                      target_names=class_names, output_dict=True)
        
        # Log final metrics
        wandb.log({
            "final/val_accuracy": final_val_acc,
            "final/val_loss": final_val_loss,
            "final/best_val_accuracy": best_val_accuracy,
        })
        
        # Log per-class metrics
        for class_name in class_names:
            wandb.log({
                f"final/{class_name}_precision": report[class_name]['precision'],
                f"final/{class_name}_recall": report[class_name]['recall'],
                f"final/{class_name}_f1": report[class_name]['f1-score']
            })
        
        # Confusion matrix
        cm = confusion_matrix(final_targets, final_preds)
        wandb.log({
            "final/confusion_matrix": wandb.plot.confusion_matrix(
                probs=None,
                y_true=final_targets,
                preds=final_preds,
                class_names=class_names
            )
        })
        
        # Feature importance
        importance_df = compute_feature_importance(
            model, X_test_tensor, X.columns.tolist(), device
        )
        
        importance_table = wandb.Table(dataframe=importance_df.head(10))
        wandb.log({"final/feature_importance_table": importance_table})
        
        # Bar chart for feature importance
        importance_data = [[name, imp] for name, imp in 
                          zip(importance_df['Feature'].head(10), 
                              importance_df['Importance'].head(10))]
        importance_wandb_table = wandb.Table(data=importance_data, 
                                            columns=["Feature", "Importance"])
        wandb.log({
            "final/feature_importance_chart": wandb.plot.bar(
                importance_wandb_table, "Feature", "Importance",
                title="Top 10 Feature Importances"
            )
        })
        
        print(f"\nFinal Results:")
        print(f"Best Validation Accuracy: {best_val_accuracy:.2f}%")
        print(f"Final Validation Accuracy: {final_val_acc:.2f}%")
        print(f"\nClassification Report:")
        print(classification_report(final_targets, final_preds, target_names=class_names))
        
        return final_val_acc

In [8]:
def setup_sweep():
    """Configure W&B sweep for hyperparameter tuning"""
    sweep_config = {
        'method': 'bayes',  # or 'grid', 'random'
        'metric': {
            'name': 'final/val_accuracy',
            'goal': 'maximize'
        },
        'parameters': {
            'learning_rate': {
                'distribution': 'log_uniform_values',
                'min': 0.0001,
                'max': 0.01
            },
            'hidden_dim': {
                'values': [64, 128, 256, 512]
            },
            'dropout_prob': {
                'distribution': 'uniform',
                'min': 0.2,
                'max': 0.6
            },
            'batch_size': {
                'values': [16, 32, 64]
            },
            'num_layers': {
                'values': [2, 3, 4]
            },
            'optimizer': {
                'values': ['Adam', 'AdamW', 'SGD']
            },
            'weight_decay': {
                'distribution': 'log_uniform_values',
                'min': 1e-6,
                'max': 1e-3
            },
            'epochs': {
                'value': 200
            },
            'patience': {
                'value': 20
            },
            'data_path': {
                'value': r"C:\Users\USER\OneDrive\Documents\draw football model datasets\combined_draw.csv"
            }
        }
    }
    return sweep_config

if __name__ == "__main__":
    # Login to W&B
    wandb.login()
    
    # Option 1: Single run with default configuration
    print("=" * 80)
    print("OPTION 1: Single Training Run")
    print("=" * 80)
    
    default_config = {
        'epochs': 200,
        'hidden_dim': 128,
        'batch_size': 32,
        'learning_rate': 0.001,
        'dropout_prob': 0.3,
        'num_layers': 2,
        'optimizer': 'Adam',
        'weight_decay': 1e-5,
        'patience': 20,
        'data_path': r"C:\Users\USER\OneDrive\Documents\draw football model datasets\combined_draw.csv"
    }
    
    # Uncomment to run single training
    # wandb.init(project="football-match-prediction", config=default_config, 
    #            name="baseline-run")
    # train_model(default_config)
    # wandb.finish()
    
    # Option 2: Hyperparameter sweep
    print("\n" + "=" * 80)
    print("OPTION 2: Hyperparameter Sweep")
    print("=" * 80)
    
    sweep_config = setup_sweep()
    sweep_id = wandb.sweep(sweep_config, project="football-match-prediction")
    
    # Run sweep
    print(f"\nStarting sweep with ID: {sweep_id}")
    print("Run the following command to start sweep agents:")
    print(f"wandb agent {sweep_id}")
    print("\nOr run directly in Python:")
    
    # Run sweep with specified number of runs
    wandb.agent(sweep_id, function=train_model, count=20)  # Run 20 experiments
    
    print("\n" + "=" * 80)
    print("Sweep completed! Check your W&B dashboard for results.")
    print("=" * 80)

wandb: Currently logged in as: adey004azeez (adey004azeez-hiipower-academy) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


OPTION 1: Single Training Run

OPTION 2: Hyperparameter Sweep
Create sweep with ID: 7talnckg
Sweep URL: https://wandb.ai/adey004azeez-hiipower-academy/football-match-prediction/sweeps/7talnckg

Starting sweep with ID: 7talnckg
Run the following command to start sweep agents:
wandb agent 7talnckg

Or run directly in Python:


wandb: Agent Starting Run: wt6hb9rk with config:
wandb: 	batch_size: 32
wandb: 	data_path: C:\Users\USER\OneDrive\Documents\draw football model datasets\combined_draw.csv
wandb: 	dropout_prob: 0.3288303064066787
wandb: 	epochs: 200
wandb: 	hidden_dim: 512
wandb: 	learning_rate: 0.008956353674387506
wandb: 	num_layers: 4
wandb: 	optimizer: Adam
wandb: 	patience: 20
wandb: 	weight_decay: 5.004714468202318e-05


Using device: cpu
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   home_team                406 non-null    object 
 1   away_team                406 non-null    object 
 2   home_score               406 non-null    int64  
 3   away_score               406 non-null    int64  
 4   Result                   406 non-null    object 
 5   home_points              406 non-null    int64  
 6   away_points              406 non-null    int64  
 7   Home Total Goals Scored  406 non-null    int64  
 8   Away Total Goals Scored  406 non-null    int64  
 9   Home Goals Conceded      406 non-null    int64  
 10  Away Goals Conceded      406 non-null    int64  
 11  Home Total Points        406 non-null    int64  
 12  Away Total Points        406 non-null    int64  
 13  id                       199 non-null    float64

Traceback (most recent call last):
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_788\2210521205.py", line 60, in train_model
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'


...

Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\Lib\site-packages\wandb\agents\pyagent.py", line 297, in _run_job
    self._function()
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_788\2210521205.py", line 60, in train_model
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

wandb: ERROR Run q51760hv errored: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'
wandb: Agent Starting Run: o07z7gxj with config:
wandb: 	batch_size: 16
wandb: 	data_path: C:\Users\USER\OneDrive\Documents\draw football model datasets\combined_draw.csv
wandb: 	dropout_prob: 0.3827730726301136
wandb: 	epochs: 200
wandb: 	hidden_dim: 256
wandb: 	learning_rate: 0.0002771712903593185
wandb: 	num_layers: 4
wandb: 	optimizer: Adam
wandb: 	patience: 20
wandb: 	weight_decay: 2.732964420111946e-06


Using device: cpu
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   home_team                406 non-null    object 
 1   away_team                406 non-null    object 
 2   home_score               406 non-null    int64  
 3   away_score               406 non-null    int64  
 4   Result                   406 non-null    object 
 5   home_points              406 non-null    int64  
 6   away_points              406 non-null    int64  
 7   Home Total Goals Scored  406 non-null    int64  
 8   Away Total Goals Scored  406 non-null    int64  
 9   Home Goals Conceded      406 non-null    int64  
 10  Away Goals Conceded      406 non-null    int64  
 11  Home Total Points        406 non-null    int64  
 12  Away Total Points        406 non-null    int64  
 13  id                       199 non-null    float64

Traceback (most recent call last):
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_788\2210521205.py", line 60, in train_model
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'


Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\Lib\site-packages\wandb\agents\pyagent.py", line 297, in _run_job
    self._function()
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_788\2210521205.py", line 60, in train_model
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

wandb: ERROR Run o07z7gxj errored: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true



Sweep completed! Check your W&B dashboard for results.
